In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
import torch

print(torch.cuda.is_available())

True


In [3]:
text_doc = "resources/images/lightfootcat/images/extracted_text.txt"
divisions = ["Dicotyledones", "Monocotyledones", "Pteridophyta", "Hepaticae", "Algae"]#
prompt = "resources/prompts/lightfootcat_prompt.yaml"

text_doc2 = "resources/images/hanbury/extracted_text.txt"
divisions2 = ["Dicotyledones", "GYMNOSPERMEAE", "Monocotyledones"]
prompt1 = "resources/prompts/hanbury_prompt.yaml"


In [4]:
with open(text_doc, "r") as f:
    extracted_text = f.read()

In [5]:
import re

extracted_text = re.sub(r"([A-Z]+)\s*(EAE|FAE|EAF)", r"\1EAE", extracted_text)
extracted_text = re.sub(r"(\w)-\n+(\w)", "\1-\2", extracted_text)

In [6]:
from lib.data_processing.text_processing import TextProcessor
text_processor = TextProcessor()
text_structure = text_processor(extracted_text, divisions=divisions, max_chunk_size=3000)
#text_blocks = text_processor.make_text_blocks(text_structure)

In [7]:
cleaned_text = text_processor.preprocess_text(extracted_text, divisions[0])

print(cleaned_text)

Dicotyledones ACERACRAE
Acer campestre L.
1 folder. Acer campestre [TA] 
Acer pseudoplatanus L.
2 folders. ot Folder 1. Acer Pseudo-Platanus [G].
i. “Maple. Bulls: [Bulstrode] Park” [JL] Folder 2. Acer Pseudo-Platanus [TA].

AMARANTHACEAE .
Amaranthus lividus L., Flora Europaea
1; 110 (1964)
1 folder. Amaranthus Blitum [TA].
i. Cites Ray’s Syn.

ii. ‘“Blite Amaranth. Aug.! It is often found on Dunghills in the neighbourhood of London. [ gather’d this on a Dunghill at Fulham near London”
{JL}. iii, “Amaranthus Blitum.
Monoec:
5. and:” [JL] ARALIACEAE
Hedera helix L.
1 folder. Hedera Helix [TA].
ARISTOLOCHIACEAE Asarum europaecum L.
1 folder. Asarum curopaeum [TA].
i. “Asarum curopaeum Lin: Asarabacca. Anglis. ex horto” [JL].
BALSAMINACEAE Impatiens noli-tangere 1.
1 folder. Impatiens Nolt me tangere [G].
BETULACEAE Alnus glutinosa (L.) Gaertn.
! folder. Betula Alnus [TA].
Betula nana L. .
1 folder. Betula nana [G].
CAMPANULACEAE Campanula latifolia L.
2 folders.
Folder 1. Companula lati

In [8]:
div_split = text_processor.split_by_divisions(extracted_text, divisions)
print(div_split)

{'Dicotyledones': ' ACERACRAE\nAcer campestre L.\n1 folder. Acer campestre [TA] \nAcer pseudoplatanus L.\n2 folders. ot Folder 1. Acer Pseudo-Platanus [G].\ni. “Maple. Bulls: [Bulstrode] Park” [JL] Folder 2. Acer Pseudo-Platanus [TA].\n\nAMARANTHACEAE .\nAmaranthus lividus L., Flora Europaea\n1; 110 (1964)\n1 folder. Amaranthus Blitum [TA].\ni. Cites Ray’s Syn.\n1957.\nii. ‘“Blite Amaranth. Aug.! It is often found on Dunghills in the neighbourhood of London. [ gather’d this on a Dunghill at Fulham near London”\n{JL}. iii, “Amaranthus Blitum.\nMonoec:\n5. and:” [JL] ARALIACEAE\nHedera helix L.\n1 folder. Hedera Helix [TA].\nARISTOLOCHIACEAE Asarum europaecum L.\n1 folder. Asarum curopaeum [TA].\ni. “Asarum curopaeum Lin: Asarabacca. Anglis. ex horto” [JL].\nBALSAMINACEAE Impatiens noli-tangere 1.\n1 folder. Impatiens Nolt me tangere [G].\nBETULACEAE Alnus glutinosa (L.) Gaertn.\n! folder. Betula Alnus [TA].\nBetula nana L. .\n1 folder. Betula nana [G].\nCAMPANULACEAE Campanula latifolia

In [9]:
sample = div_split["Dicotyledones"]

finds = re.finditer(text_processor.family_regex, sample)

find_matches = [i for i in finds]
text_chunks = []

for idx, i in enumerate(find_matches):
    match = re.sub(r"[.\n\t,]*\s*([A-Z]+)\s*[.\n\t,]*", r"\1", i.group())
    start = i.end()
    end = find_matches[idx+1].start() if idx+1 < len(find_matches) else None
    text_chunk = sample[start:end] if end else sample[start:]
    text_chunks.append(dict(family=match, text=text_chunk))

In [10]:
text_chunks

[{'family': 'ACERACRAE',
  'text': 'Acer campestre L.\n1 folder. Acer campestre [TA] \nAcer pseudoplatanus L.\n2 folders. ot Folder 1. Acer Pseudo-Platanus [G].\ni. “Maple. Bulls: [Bulstrode] Park” [JL] Folder 2. Acer Pseudo-Platanus [TA]'},
 {'family': 'AMARANTHACEAE',
  'text': '.\nAmaranthus lividus L., Flora Europaea\n1; 110 (1964)\n1 folder. Amaranthus Blitum [TA].\ni. Cites Ray’s Syn.\n1957.\nii. ‘“Blite Amaranth. Aug.! It is often found on Dunghills in the neighbourhood of London. [ gather’d this on a Dunghill at Fulham near London”\n{JL}. iii, “Amaranthus Blitum.\nMonoec:\n5. and:” [JL]'},
 {'family': 'ARALIACEAE',
  'text': 'Hedera helix L.\n1 folder. Hedera Helix [TA]'},
 {'family': 'ARISTOLOCHIACEAE',
  'text': 'Asarum europaecum L.\n1 folder. Asarum curopaeum [TA].\ni. “Asarum curopaeum Lin: Asarabacca. Anglis. ex horto” [JL]'},
 {'family': 'BALSAMINACEAE',
  'text': 'Impatiens noli-tangere 1.\n1 folder. Impatiens Nolt me tangere [G]'},
 {'family': 'BETULACEAE',
  'text': '

In [11]:

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import gc

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", temperature=0.0)

/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only u

In [18]:
def split(model, tokenizer, text):

    message = (
        "You are an expert taxonomist. You are given the output of an OCR model. The output is from a botanical catalogue.\n "
        "A record contains the species name followed by the folder information about the species. \n"
        "The text is a collection of records. \n"
        "Seperate the text into different paragraphs, keep each record in the same paragraph with respect to context. \n"
        "One seperated, return each paragraph as a json object with the following key: paragraph. \n"
        "Ensure nothing is removed from the text. \n"
        "DO NOT GENERATE ANY EXTRA TEXT. \n"
        "Only return the list of JSON objects \n"
        f"Ocr output: \n\n {text}"
    )
    
    conversation = [{"role": "user", "content": message}]


    # format and tokenize the tool use prompt 
    inputs = tokenizer.apply_chat_template(
                conversation,
                return_dict=True,
                return_tensors="pt",
    )

    print("Inputs generated")
    print("Input size:", inputs.input_ids.shape)
    max_tokens = inputs.input_ids.shape[1]
    print("Max tokens:", max_tokens)
    print("========================================================================")

    inputs.to(model.device)

    outputs = model.generate(**inputs, max_new_tokens=max_tokens)
    print("Outputs generated")
    generated_ids = [
                output_ids[len(input_ids) :]
                for input_ids, output_ids in zip(inputs.input_ids, outputs)
            ]
    final_out = tokenizer.batch_decode(generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    print("Final Output Generated")
    #print(final_out[0])

    return final_out[0]

In [19]:
print(repr(text_chunks[0]["text"]))

'Acer campestre L.\n1 folder. Acer campestre [TA] \nAcer pseudoplatanus L.\n2 folders. ot Folder 1. Acer Pseudo-Platanus [G].\ni. “Maple. Bulls: [Bulstrode] Park” [JL] Folder 2. Acer Pseudo-Platanus [TA]'


In [20]:
%%time
new_chunks = []
sample_size = 5
big_chunks = []
for i in text_chunks:
    family = i["family"]
    text = i["text"]
    if len(text) > 1000:
        big_chunks.append(dict(family=family, text=text))
        continue
    elif sample_size <= 0:
        break
    print("Processing Family:", family)
    split_text = split(model, tokenizer, text)
    print("Text Splitted")
    new_chunks.append(dict(family=family, text=text, split_text=split_text))
    print("Chunk added")
    sample_size -= 1
    

/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/mnt/apps/users/ikarunak/conda/envs/lcat/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Processing Family: ACERACRAE
Inputs generated
Input size: torch.Size([1, 237])
Max tokens: 237


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Outputs generated
Final Output Generated
Text Splitted
Chunk added
Processing Family: AMARANTHACEAE
Inputs generated
Input size: torch.Size([1, 282])
Max tokens: 282


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Outputs generated
Final Output Generated
Text Splitted
Chunk added
Processing Family: ARALIACEAE
Inputs generated
Input size: torch.Size([1, 164])
Max tokens: 164


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Outputs generated
Final Output Generated
Text Splitted
Chunk added
Processing Family: ARISTOLOCHIACEAE
Inputs generated
Input size: torch.Size([1, 200])
Max tokens: 200


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Outputs generated
Final Output Generated
Text Splitted
Chunk added
Processing Family: BALSAMINACEAE
Inputs generated
Input size: torch.Size([1, 172])
Max tokens: 172
Outputs generated
Final Output Generated
Text Splitted
Chunk added
CPU times: user 12 s, sys: 33.3 ms, total: 12 s
Wall time: 12 s
